In [ ]:
import matplotlib.pyplot as plt
import os
import sys

sys.path.append('..')
os.environ.update(dict(CUDA_VISIBLE_DEVICES='3'))

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from tqdm import tqdm

from torch.distributions import MultivariateNormal
from models.Res import ResNet, resnet50

from analysis import *

def hook(module, args, output):
    module._output = output.data
    return output

class ResNetWrapper(nn.Module):
    def __init__(self, model):
        super(ResNetWrapper, self).__init__()
        self.model = model
        self.model.requires_grad_(False)
        self.image = torch.autograd.Variable(torch.randn(1, 3, 224, 224).cuda(), requires_grad=True)
    
    def forward(self):
        return self.model(self.image)

In [ ]:
model = resnet50(pretrained=True).cuda()
model = ResNetWrapper(model)
model.eval()

model.model.layer3.register_forward_hook(hook)